In [19]:
from scipy import stats
import numpy as np
import pandas as pd


from scipy.stats import norm, t, kstest, shapiro
import statsmodels.api as sm
from matplotlib import pyplot as plt


import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

## Задача

Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

**Mobile Games AB Testing with Cookie Cats**



In [7]:
df = pd.read_csv('cookie_cats2.csv')
df.head()
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


Преобразуем колонки sum_gamerounds, retention_1, и retention_7 для уменьшения объема занимаемой памяти.

In [12]:
df = df.astype({'sum_gamerounds': np.uint16})
df = df.astype({'retention_1': np.uint8})
df = df.astype({'retention_7': np.uint8})

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  uint16
 3   retention_1     90189 non-null  uint8 
 4   retention_7     90189 non-null  uint8 
dtypes: int64(1), object(1), uint16(1), uint8(2)
memory usage: 1.7+ MB


Проверим есть ли некорректные данные

In [15]:
df.userid.value_counts().sort_values()

userid
116        1
488        1
540        1
1066       1
1444       1
          ..
9994217    1
9997049    1
9996999    1
9974108    1
9999861    1
Name: count, Length: 90189, dtype: int64

In [17]:
df.duplicated(subset=['userid', 'version']).sum()

0

Вроде по данным все нормально

In [14]:
df.version.value_counts().sort_values()

version
gate_30    44700
gate_40    45489
Name: count, dtype: int64

Данные распределены не нормально. Применим не парметрический тест Манна-Уитни.

Нулевая гипотеза - статически значимое различие есть.

Работаем с **retention_1**

In [20]:
gr1 = df[df['version'] == 'gate_30']['retention_1']
gr2 = df[df['version'] == 'gate_40']['retention_1']

stats.mannwhitneyu(gr1, gr2)

MannwhitneyuResult(statistic=1022682813.0, pvalue=0.07441128639919536)

p-value больше 0.05 --> 
Таким образом делам вывод о том, что статистической значимой разницы между группами нет.
Но это только ретеншион за один день.

Работаем с **retention_7**

In [21]:
gr1 = df[df['version'] == 'gate_30']['retention_7']
gr2 = df[df['version'] == 'gate_40']['retention_7']

stats.mannwhitneyu(gr1, gr2)

MannwhitneyuResult(statistic=1025017239.0, pvalue=0.001554344685704005)

p-value меньше 0.05 --> 
Таким образом делам вывод о том, что статистическая значимая разница между группами есть.

In [24]:
df.query('version == "gate_30"').describe()


,userid,sum_gamerounds,retention_1,retention_7
count,4.470000e+04,44700.000000,44700.000000,44700.000000
mean,4.987564e+06,52.456264,0.448188,0.190201
std,2.881026e+06,256.716423,0.497314,0.392464
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.505469e+06,5.000000,0.000000,0.000000
50%,4.983631e+06,17.000000,0.000000,0.000000
75%,7.481497e+06,50.000000,1.000000,0.000000
max,9.999710e+06,49854.000000,1.000000,1.000000


In [23]:
df.query('version == "gate_40"').describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.548900e+04,45489.000000,45489.000000,45489.000000
mean,5.009073e+06,51.298776,0.442283,0.182000
std,2.885496e+06,103.294416,0.496663,0.385849
min,3.770000e+02,0.000000,0.000000,0.000000
25%,2.517171e+06,5.000000,0.000000,0.000000
50%,5.007329e+06,16.000000,0.000000,0.000000
75%,7.510762e+06,52.000000,1.000000,0.000000
max,9.999861e+06,2640.000000,1.000000,1.000000


Полагаю, что retention_7 — это метрика, отвечающая на вопрос, сколько людей вернулись за семь дней.

Учитывая среднее значение можем сделать вывод и рекомендацию менеджеру, что идея переноса на сороковые ворота была неудачная.
Желательно не использовать данное новшество.